In [ ]:
import numpy as np
import pandas as pd
import pickle
import re, nltk, spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from gensim.utils import simple_preprocess
from pprint import pprint
from nltk.corpus import stopwords
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../data/cfpb-selected.csv')
df.head()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/18/21,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,I previously asked the credit bureaus for a in...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",PA,191XX,None,Consent provided,Web,03/18/21,Closed with explanation,Yes,NaN,4225218
1,03/23/21,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Investigation took more than 30 days,"XXXX, Experian and XXXX, FHA Resource Center, ...",Company has responded to the consumer and the ...,Experian Information Solutions Inc.,UT,840XX,None,Consent provided,Web,03/23/21,Closed with explanation,Yes,NaN,4239496
2,04/07/21,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Was not notified of investigation status or re...,On XX/XX/2021 sent a letter regarding inaccura...,None,"EQUIFAX, INC.",FL,322XX,None,Consent provided,Web,04/07/21,Closed with explanation,Yes,NaN,4278158
3,04/09/21,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,I received an email about a decrease in my con...,None,"EQUIFAX, INC.",GA,None,None,Consent provided,Web,04/09/21,Closed with explanation,Yes,NaN,4283212
4,03/25/21,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,There is a fraudulent bank account that is rep...,Company has responded to the consumer and the ...,"Fidelity National Information Services, Inc. (...",NY,114XX,None,Consent provided,Web,03/25/21,Closed with explanation,Yes,NaN,4245653


In [3]:
df['Issue'].value_counts()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Incorrect information on your report                                                18175
Problem with a credit reporting company's investigation into an existing problem    14768
Improper use of your report                                                          2871
Attempts to collect debt not owed                                                    2564
Trouble during payment process                                                       2543
Managing an account                                                                  1967
Problem with a purchase shown on your statement                                      1644
Struggling to pay mortgage                                                            882
Closing on a mortgage                                                                 689
Unable to get your credit report or credit score                                      652
Fees or interest                                                                      620
Closing an

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49491 entries, 0 to 49490
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date received                 49491 non-null  object 
 1   Product                       49491 non-null  object 
 2   Sub-product                   49491 non-null  object 
 3   Issue                         49491 non-null  object 
 4   Sub-issue                     49491 non-null  object 
 5   Consumer complaint narrative  49491 non-null  object 
 6   Company public response       49491 non-null  object 
 7   Company                       49491 non-null  object 
 8   State                         49491 non-null  object 
 9   ZIP code                      49491 non-null  object 
 10  Tags                          49491 non-null  object 
 11  Consumer consent provided?    49491 non-null  object 
 12  Submitted via                 49491 non-null  object 
 13  D

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
df = df.drop(['Date received', 'Sub-issue', 'Company public response', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?', 'Submitted via', 'Company response to consumer', 'Timely response?', 'Consumer disputed?', 'Complaint ID'], axis=1)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
data = df['Consumer complaint narrative'].tolist()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]
data = [sent.lower() for sent in data] #lowecase removal
data = [re.sub(r'\s+', ' ', sent) for sent in data]

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
# Tokenize
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(data))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'xxxx', 'xx', 'xxxxxx', 'xxxxxxxx', 'xxxxx'])
for key in df['Company'].value_counts().keys():
    stop_words.extend([key.lower()])

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data_words = remove_stopwords(data_words)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN','VERB']) #select noun and verb

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10, 
                             token_pattern='[a-zA-Z0-9]{3,}',  
                            )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
search_params = {'n_components': [30], 'learning_decay': [0.7]}
# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0)
# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)
# Do the Grid Search
model.fit(data_vectorized)

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


GridSearchCV(estimator=LatentDirichletAllocation(learning_method='online',
                                                 learning_offset=50.0,
                                                 max_iter=5, random_state=0),
             param_grid={'learning_decay': [0.7], 'n_components': [30]})

In [16]:
# Best LDA model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Best Model's Params:  {'learning_decay': 0.7, 'n_components': 30}
Best Log Likelihood Score:  -3270141.410222464
Model Perplexity:  331.40886003844554


In [17]:
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ["Doc" + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return "color: {col}".format(col=color)
def make_bold(val):
    weight = 700 if val > .1 else 400
    return "font-weight: {weight}".format(weight=weight)

df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,dominant_topic
Doc0,0.000000,0.000000,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.230000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.380000,0.000000,0.000000,0.000000,26
Doc1,0.060000,0.000000,0.060000,0.230000,0.000000,0.070000,0.000000,0.000000,0.000000,0.010000,0.000000,0.050000,0.000000,0.000000,0.000000,0.050000,0.000000,0.180000,0.200000,0.000000,0.000000,0.050000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3
Doc2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.980000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7
Doc3,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.700000,0.030000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.040000,16
Doc4,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.040000,0.000000,0.030000,0.000000,0.000000,0.000000,0.080000,0.000000,0.000000,0.100000,0.400000,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.140000,0.050000,16
Doc5,0.030000,0.140000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000,0.270000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.210000,0.000000,8
Doc6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.260000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140000,0.000000,0.000000,0.110000,19
Doc7,0.050000,0.000000,0.000000,0.000000,0.100000,0.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.280000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.110000,5
Doc8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.170000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.620000,0.000000,28
Doc9,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,0.370000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.110000,0.230000,0.000000,0.010000,0.000000,0.000000,0.090000,0.000000,0.010000,0.000000,0.080000,0.010000,11


In [18]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic Num,Num Documents
0,28,11312
1,18,6578
2,8,3636
3,11,3308
4,7,3151
5,10,2710
6,21,2175
7,16,2084
8,15,2069
9,5,1725


In [19]:
def show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,complaint,file,bankruptcy,cfpb,court,attorney,include,submit,union,case,continue,state,attach,matter,company
Topic 1,information,letter,verify,request,credit,advise,send,process,investigation,receive,dispute,contact,bureaus,remove,submit
Topic 2,company,respond,require,dept,day,comply,law,report,failure,prevent,demand,complaint,record,agreement,regulation
Topic 3,fraud,live,scam,pnc,seller,apartment,rent,child,commit,wife,person,cell,mastercard,pin,protect
Topic 4,provide,receive,dispute,email,service,refund,request,bank,claim,contact,return,issue,date,customer,purchase
Topic 5,mortgage,loan,pay,insurance,escrow,home,company,property,refinance,tax,taxis,amount,rate,lender,closing
Topic 6,well,fargo,account,acct,open,charge,request,balance,receive,make,amount,dispute,credit,act,write
Topic 7,day,item,credit,report,dispute,file,receive,delete,regard,datum,information,response,demand,investigation,investigate
Topic 8,debt,collection,account,provide,report,credit,contract,company,signature,request,proof,allege,creditor,owe,validation
Topic 9,rule,people,citizen,release,recall,protect,group,divorce,evidence,research,burden,select,record,reinstate,reveal


In [20]:
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,aargon,abandon,abide,ability,able,absence,absolve,abundance,abuse,abuser,...,yard,year,yell,yesterday,yield,york,yrs,zero,zip,zone
Topic0,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,77.093440,6.429104,...,0.033333,318.977602,0.033333,0.033333,0.033333,92.114078,0.033333,0.033333,0.033333,0.033333
Topic1,0.033333,0.033333,0.033340,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,...,0.033333,0.033333,0.033333,0.033333,0.033333,0.033343,0.033333,0.033333,0.033333,0.033333
Topic2,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,...,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333
Topic3,0.033333,26.294625,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,...,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033335,0.033333,0.033333,0.033333
Topic4,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,...,0.033333,1.902979,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,32.059986,0.033334


In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
# Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
# Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
# Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), 1:14].values.tolist()
    #topic_guess = df_topic_keywords.iloc[np.argmax(topic_probability_scores), Topics]
    return topic, topic_probability_scores
# Predict the topic
mytext = 'My complaint is with Carrington Mortgage Services. The company is charging property inspection payments of XXXX along with mortgage of XXXX the dates are from XX/XX/XXXX until present, also attorney fees. This makes my payments double. Carrington is also Charges a fee of XXXX unknown.'
topic, prob_scores = predict_topic(text = mytext)
print(topic)

In [21]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\ASUS\anaconda3\envs\marketplace\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
18     119.070190   64.655502       1        1  13.015165
28      54.382130  109.799164       2        1  10.244182
11    -161.423965 -103.744957       3        1   6.843346
8     -135.370758   36.673065       4        1   5.910848
16     177.317215  -66.872452       5        1   5.133877
4      -16.436356  142.453796       6        1   4.865021
1       31.052383  204.777573       7        1   4.421238
5      -25.024384 -186.083466       8        1   4.322349
7      -68.157486   20.934038       9        1   3.849776
15      -3.751695    4.961509      10        1   3.844218
17    -114.470009 -179.605469      11        1   3.734517
19    -110.729126  -40.476776      12        1   3.664746
21      42.552418  -42.789406      13        1   3.365322
10     -10.661536   71.922211      14        1   3.337480
24     112.652969  -16.279951      15        1   3.065812
29    -195.499146  -25.853945      16        1   3.010481
26    -207.165924   66.070419      17        1   2.977316
0       10.217649 -110.752060      18        1   2.139745
25     -81.255081   96.477776      19        1   1.890604
12     -77.253441  192.755219      20        1   1.855941
6      182.353027  110.412880      21        1   1.640470
2       89.766510  -93.997673      22        1   1.597463
27     -36.483528  -52.542774      23        1   1.199276
14     -72.052170 -112.144821      24        1   0.912610
23     144.822769 -150.811508      25        1   0.876211
3     -148.475357  135.678162      26        1   0.851289
20     189.811615   17.382956      27        1   0.453397
9       56.555355   33.755768      28        1   0.439435
22      58.988537 -178.376541      29        1   0.358639
13     110.556717  164.236694      30        1   0.179227, topic_info=         Term          Freq         Total Category  logprob  loglift
24    account  96788.000000  96788.000000  Default  30.0000  30.0000
2466  payment  38862.000000  38862.000000  Default  29.0000  29.0000
825    credit  99501.000000  99501.000000  Default  28.0000  28.0000
2928   report  83025.000000  83025.000000  Default  27.0000  27.0000
476      card  20241.000000  20241.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
2730    query     51.982107     52.964360  Topic30  -4.5576   6.3056
1632      hoa     51.695658     52.677912  Topic30  -4.5631   6.3054
1387   finger     51.152621     52.134874  Topic30  -4.5737   6.3053
3560     tire     50.226058     51.208311  Topic30  -4.5920   6.3049
4        able     50.048171     51.030424  Topic30  -4.5955   6.3048

[1879 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2         5  0.994210       abide
4        30  0.979808        able
6        21  0.965568     absolve
11       13  0.929347    acccount
15       24  0.998093  acceptance
...     ...       ...         ...
3888     23  0.002923        year
3888     25  0.001357        year
3890      1  0.921646   yesterday
3890     11  0.023861   yesterday
3890     14  0.053688   yesterday

[4259 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 29, 12, 9, 17, 5, 2, 6, 8, 16, 18, 20, 22, 11, 25, 30, 27, 1, 26, 13, 7, 3, 28, 15, 24, 4, 21, 10, 23, 14])

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
def similar_documents(text, doc_topic_probs, documents = data, nlp=nlp, top_n=5, verbose=False):
    topic, x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    if verbose:        
        print("Topic KeyWords: ", topic)
        print("Topic Prob Scores of text: ", np.round(x, 1))
        print("Most Similar Doc's Probs:  ", np.round(doc_topic_probs[doc_ids], 1))
    return doc_ids, np.take(documents, doc_ids)

In [ ]:
mytext = ['I have an issue with closing my bank account']
doc_ids, docs = similar_documents(text=mytext, doc_topic_probs=lda_output, documents = data, top_n=5, verbose=True)
print('\n', docs[0][:500])